In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

from functions import winsor, corr_uniform, sPCAest
from scipy.ndimage import shift
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA, KernelPCA, SparsePCA, FastICA
from sklearn.preprocessing import StandardScaler




# Weak factor case

In [2]:
def generate_data(n, T, N, h_steps=1, heteroskedastic=False, psi_max=None, rho=None):
    # Check if weak or strong factor model
    if n == N:
        # Strong factor model, require psi_max and rho
        assert psi_max is not None
        assert rho is not None

    # Generate y_{t + h} = g_t + e_{t + h}
    # e_{t + h} ~ N(0, 1)
    # g_t ~ N(0, 1)
    # X_t,i = g_t*phi_i + h_t*psi_i + u_t,i

    # Set T to T + h to account for lag
    T_plus_h = T + h_steps

    # Generate g_t
    g = np.zeros(T_plus_h)
    g = np.random.normal(0, 1, T_plus_h)

    # Generate h_t
    h = np.zeros(T_plus_h)
    h = np.random.normal(0, 1, T_plus_h)
    
    # Generate e_t+h
    e = np.zeros(T_plus_h)
    e = np.random.normal(0, 1, T_plus_h)

    # Generate y_{t+h}
    y_h = np.zeros(T_plus_h)
    y_h = g + e

    # Generate y_t
    y_t = np.zeros(T_plus_h)
    y_t[h_steps:T_plus_h] = y_h[:T]

    phi = np.zeros(N)
    psi = np.zeros(N)

    if n < N:
        # Generate phi_i (Nx1) with n < N nonzero elements
        phi[:n] = np.random.uniform(0, 1, n)

        # Generate psi_i (Nx1) with n < N nonzero elements
        psi[:n] = np.random.uniform(0, 1, n)

        # Generate idiosyncratic error's variances
        variance_u = np.random.uniform(0, 1, N)        
    elif n == N:
        # Phi is U(0, phi_max)
        psi = np.random.uniform(0, psi_max, N)

        # Draw correlated phi and sigma_u
        # Generate correlation matrix for phi and sigma_u
        phi, variance_u = corr_uniform(rho=rho*1.1, size=N)

    u = np.zeros((T, N))
    # Generate u_t,i depending on whether the model is heteroskedastic or not
    if heteroskedastic:
        for t in range(T):
            scale = np.random.uniform(0.5, 1.5)
            u[t, :] = np.random.normal(0, variance_u * scale, N)
    else:
        # Generate u
        u = np.random.multivariate_normal(mean=np.zeros(N), cov=np.diag(variance_u), size=T)

    # Drop first h rows
    y_h = y_h[h_steps:]
    y_t = y_t[h_steps:]
    e = e[h_steps:]
    g = g[h_steps:]
    h = h[h_steps:]

    X = g[:, np.newaxis]*phi[np.newaxis, :] + h[:, np.newaxis]*psi[np.newaxis, :] + u

    results = {'y_t':y_t,
               'y_h':y_h,
                'X':X,
                'g':g,
                'e':e,
                }
    
    return results

# Testing
results = generate_data(10, T=250, N=500, h_steps=1)
y_t = results['y_t']
y_h = results['y_h']
X = results['X']
e = results['e']
g = results['g']

print(y_t.shape)
print(y_h.shape)
print(X.shape)

print('y_t: ', y_t[:10])
print('y_h: ', y_h[:10])
print()
print('e: ', e[:10])
print('g: ', g[:10])


(250,)
(250,)
(250, 500)
y_t:  [ 0.05987219  0.94242811 -1.90025016 -1.00953636 -1.81306809 -0.35928276
  0.53298677 -1.08345965  1.29815511  2.40004253]
y_h:  [ 0.94242811 -1.90025016 -1.00953636 -1.81306809 -0.35928276  0.53298677
 -1.08345965  1.29815511  2.40004253  0.43963098]

e:  [ 0.28987748 -1.45974756 -0.44667645 -1.82236392 -0.27448564 -0.18658702
 -0.06461369  1.68464557  0.33557557  1.00175189]
g:  [ 0.65255063 -0.4405026  -0.56285991  0.00929583 -0.08479713  0.71957379
 -1.01884596 -0.38649046  2.06446697 -0.56212091]


In [4]:
def predict_pca(X, y_h, y_t, h_steps=1, nfac=2, method="sPCA", start_test=200):
    pca = PCA(n_components=nfac)
    reg = LinearRegression()

    predicted = np.zeros((1, 50 - h_steps - 1))
    error_mat = np.zeros((1, 50 - h_steps - 1))
    normalize = StandardScaler(with_mean=True, with_std=True)

    for t in range(50 - h_steps - 1):
        if t % 10 == 0:
            print("Period {}".format(t))

        # Split data into training and testing sets
        # Training set is the first 200 + t observations
        # Testing set is the last 50 - t observations
        idx_split = start_test + t
        
        X_train = X[:idx_split]
        y_train = y_h[:idx_split]
        y_test = y_t[idx_split]

        if method == "sPCA":
            # Estimate the parameters of the model using sPCAest
            # The function sPCAest is defined in functions.py
            factors, _ = sPCAest(y_train, X_train, nfac,[0, 100], h_steps)
        elif method == "PCA":
            X_normalized = normalize.fit_transform(X_train)
            factors = pca.fit_transform(X_normalized)

        reg.fit(factors[:-h_steps,:], y_train[:-h_steps])
        
        # Predict y_{t+h} using the estimated parameters
        y_pred = reg.predict(factors[-1].reshape(1, -1))

        # Add predicted value to the predicted vector
        predicted[0, t] = y_pred
        
        # Compute the error
        error_mat[0, t] = y_pred - y_test

    return error_mat, predicted


def simulate_PCA(n=10, T=250, N=500, h_steps=1, R=10, nfac=2, method="sPCA", heteroskedastic=False, psi_max=None, rho=None):
    # Initialize the MSE vector
    predicted = np.zeros((R, 50 - h_steps - 1))
    error_mat = np.zeros((R, 50 - h_steps - 1))

    start_test = 200

    for r in range(R):
        print('r: ', r)
        variables = generate_data(n=n, T=T, h_steps=h_steps, N=N, heteroskedastic=heteroskedastic, psi_max=psi_max, rho=rho)

        # y_t contains the values of y at time t, and y_h contains the values of y at time t+h
        # X contains the values of X at time t
        y_h = variables['y_h']
        y_t = variables['y_t']
        X = variables['X']
        
        # Loop for expanding window estimation
        # The initial window size is 200 observations
        # The window size is increased by 1 observation at each iteration
        # The window size is increased until it reaches 250 observations
        errors, predictions = predict_pca(X, y_h, y_t, h_steps, nfac, method, start_test)
            
    return error_mat, predicted

In [22]:
import warnings
warnings.filterwarnings('ignore')

# Run the simulationc:\Users\Vincent\Anaconda3New\envs\thesis\lib\site-packages\sklearn\decomposition\_fastica.py:729
errors, predictions = simulate_PCA(n=50, T=250, N=500, h_steps=1, R=20, nfac=2, method="sPCA", heteroskedastic=False)

r:  0
Period 0
Period 10
Period 20
Period 30
Period 40
r:  1
Period 0
Period 10
Period 20
Period 30
Period 40
r:  2
Period 0
Period 10
Period 20
Period 30
Period 40
r:  3
Period 0
Period 10
Period 20
Period 30
Period 40
r:  4
Period 0
Period 10
Period 20
Period 30
Period 40
r:  5
Period 0
Period 10
Period 20
Period 30
Period 40
r:  6
Period 0
Period 10
Period 20
Period 30
Period 40
r:  7
Period 0
Period 10
Period 20
Period 30
Period 40
r:  8
Period 0
Period 10
Period 20
Period 30
Period 40
r:  9
Period 0
Period 10
Period 20
Period 30
Period 40
r:  10
Period 0
Period 10
Period 20
Period 30
Period 40
r:  11
Period 0
Period 10
Period 20
Period 30
Period 40
r:  12
Period 0
Period 10
Period 20
Period 30
Period 40
r:  13
Period 0
Period 10
Period 20
Period 30
Period 40
r:  14
Period 0
Period 10
Period 20
Period 30
Period 40
r:  15
Period 0
Period 10
Period 20
Period 30
Period 40
r:  16
Period 0
Period 10
Period 20
Period 30
Period 40
r:  17
Period 0
Period 10
Period 20
Period 30
Period 40
r:

In [ ]:
mse_vec = (errors**2).mean(axis=1)
median_mse = np.median(mse_vec)
mean_mse = mse_vec.mean()
print("Median MSE is {}".format(median_mse))
print("Mean MSE is {}".format(mean_mse))
print("MSE is {}".format(mse_vec))

## Plotting

In [ ]:
# Plot erros of the first run of the simulation
plt.plot((predictions[0,:]))
plt.title("Predicted values of the first run of the simulation")
# Include tick at every other integer
plt.xticks(np.arange(0, 50, 2))
plt.show()

# Plot the histogram of  mean squared error over the different runs
plt.hist(mse_vec, bins=40)
plt.title("Histogram of mean squared error over the different runs")
plt.show()
